In [ ]:
import pandas as pd

ctl = pd.read_csv("celltype_lipidomes.csv", index_col=0)
ctl = ctl.iloc[2:,:]
ctl

In [ ]:
import numpy as np

fitznerlips = ctl.index
astro = np.nanmean(ctl[['Astrocytes1', 'Astrocytes2',
       'Astrocytes3']].astype(float),axis=1)

mgl = np.nanmean(ctl[['Microglia1', 'Microglia2', 'Microglia3']].astype(float),axis=1)

oli = np.nanmean(ctl[['OligodendrocytesDIV11', 'OligodendrocytesDIV12',
       'OligodendrocytesDIV13', 'OligodendrocytesDIV251',
       'OligodendrocytesDIV252', 'OligodendrocytesDIV253',
       'OligodendrocytesDIV41', 'OligodendrocytesDIV42',
       'OligodendrocytesDIV43']].astype(float),axis=1)

neu = np.nanmean(ctl[['NeuronsDIV101', 'NeuronsDIV102', 'NeuronsDIV103', 'NeuronsDIV161',
       'NeuronsDIV162', 'NeuronsDIV163', 'NeuronsDIV51', 'NeuronsDIV52',
       'NeuronsDIV53']].astype(float),axis=1)

aveprof = pd.DataFrame([astro, mgl, oli, neu], columns=ctl.index, index = ["astro", "mgl", "oli", "neu"]).T
aveprof.fillna(0, inplace=True)
aveprof

In [ ]:
names = pd.read_csv("goslinfitzner_celltyoes.tsv", sep="\t")
namesgoslin = names[['Original Name', 'Species Name']]
namesgoslin.index = namesgoslin['Original Name']

aveprof.index = namesgoslin.loc[:,'Species Name']
aveprof

In [ ]:
tmp = pd.read_hdf("20241104_FullyAnnotatedLBA.h5ad", key="table")
tmp

In [ ]:
tree = pd.read_hdf("tree_seeded_deep-Copy1.h5ad", key="table")
tmp = tmp.loc[tree.index,:]
tmp

In [ ]:
centroids = tmp.iloc[:, :1400].groupby(tree['cluster']).mean()
centroids

In [ ]:
incommon = np.intersect1d(aveprof.index.astype(str), centroids.columns.astype(str))
len(incommon) # 173 lipids to be used for deconvolution

In [ ]:
aveprof = aveprof.loc[incommon,:]
aveprof = aveprof.groupby(aveprof.index).mean()
aveprof = aveprof.T
aveprof = (aveprof - aveprof.min()) / (aveprof.max() - aveprof.min())
aveprof

In [ ]:
centroids = centroids.loc[:, aveprof.columns]
centroids = (centroids - centroids.min()) / (centroids.max() - centroids.min())

aveprof_pinv = np.linalg.pinv(aveprof.values)

matrix_X = np.dot(centroids.values, aveprof_pinv)

matrix_X_df = pd.DataFrame(matrix_X, index=centroids.index, columns=aveprof.index)
matrix_X_df = (matrix_X_df.T / matrix_X_df.sum(axis=1)).T
matrix_X_df